In [11]:
%load_ext autoreload
%autoreload 2

import os, sys
import copy
import socket
from tqdm import tqdm
import torch
import pickle
from torch import optim
from torch.utils.tensorboard import SummaryWriter

#!pip install networkx matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../")))
from libs.topology_manager import *
from libs import fl, nn, agg, data, log

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
#log.init("info", "federated.log")
#log.init("debug", "flkafka.log")

In [13]:
adj_mat = np.array([[0,1,1,1,1,1],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0],
             [1,0,0,0,0,0], 
             [1,0,0,0,0,0]])

node_types = {'aggregator': [0,1], 'trainer': [2,3], 'broadcaster': [4,5]}

di_graph = nx.DiGraph(adj_mat)
#nx.draw(di_graph)

In [14]:
class FedArgs():
    def __init__(self):
        self.num_clients = len(adj_mat)
        self.epochs = 10
        self.local_rounds = 1
        self.client_batch_size = 32
        self.test_batch_size = 128
        self.learning_rate = 1e-4
        self.weight_decay = 1e-5
        self.cuda = False
        self.seed = 1
        self.topic = 'pyflx-c'
        self.server_topic = "pyflx-cs"
        self.tb = SummaryWriter('../out/runs/flkafka', comment="Mnist Distributed Federated training")

fedargs = FedArgs()

In [15]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [16]:
host = socket.gethostname()
server = host + "(server)"
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]
ctp = CentralizedTopology(adj_mat, server, clients, node_types)

In [17]:
#Initialize Global and Client models
global_model = nn.ModelMNIST()
client_models = {client: copy.deepcopy(global_model) for client in clients}

# Function for training
def train_model(_model, train_loader, fedargs, device):
    model, loss = fl.client_update(_model,
                                train_loader,
                                fedargs.learning_rate,
                                fedargs.weight_decay,
                                fedargs.local_rounds,
                                device)
    model_update = agg.sub_model(_model, model)
    return model_update, model, loss

In [18]:
# Load MNIST Data to clients
train_data, test_data = data.load_dataset("mnist")
clients_data = data.split_data(train_data, clients)
client_train_loaders, client_test_loaders = data.load_client_data(clients_data, fedargs.client_batch_size, 0.2, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

clients_info = {
        client: {"train_loader": client_train_loaders[client],
                 "test_loader": client_test_loaders[client]}
        for client in clients
    }

In [19]:
import asyncio
import time

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)

    return wrapped

@background
def process(client, epoch, ctp, model, train_loader, test_loader, fedargs, device):
    log.info("Processing Client {}".format(client))

    # Consume and Average, epoch passed is actually prev epoch, for which we want to consume updates
    rcvd_models = ctp.consume_model(client, fedargs.server_topic, model, epoch)
    log.info("Client {} received {} model update(s) from {}".format(client, len(rcvd_models), list(rcvd_models.keys())))
    if len(rcvd_models) != 0:
        model = fl.federated_avg(rcvd_models)

    # Train  
    model_update, model, loss = train_model(model, train_loader, fedargs, device)
    
    ctp.produce_model(client, fedargs.topic, model_update, epoch)

    # Plot and Log
    for local_epoch, loss in enumerate(list(loss.values())):
        fedargs.tb.add_scalars("Training Loss/" + client, {str(epoch): loss}, str(local_epoch + 1))

    log.jsondebug(loss, "Epoch {} of {} : Federated Training loss, Client {}".format(epoch, fedargs.epochs, client))
    log.modeldebug(model, "Epoch {} of {} : Client {} Update".format(epoch, fedargs.epochs, client))

    # Test
    test_output = fl.eval(model, test_loader, device)
    fedargs.tb.add_scalar("Accuracy/" + client, test_output["accuracy"], epoch)
    log.jsoninfo(test_output, "Test Outut after Epoch {} of {} for Client {}".format(epoch, fedargs.epochs, client))

In [ ]:
import time
start_time = time.time()

# Federated Training
for _epoch in tqdm(range(fedargs.epochs)):

    epoch = _epoch + 1
    log.info("Federated Training Epoch {} of {}".format(epoch, fedargs.epochs))
    
    if epoch > 1:
        # Server, Consume and Average, epoch passed is actually prev epoch, for which we want to consume updates
        client_model_updates = ctp.consume_model(server, fedargs.topic, global_model, epoch - 1)
        log.info("Server received {} model update(s) from {}".format(len(client_model_updates), list(client_model_updates.keys())))
        if len(client_model_updates) != 0:
            global_model = fl.federated_avg(client_model_updates, global_model)
        
        # Gloabal Test
        log.modeldebug(global_model, "Epoch {} of {} : Server Update".format(epoch, fedargs.epochs))
        global_test_output = fl.eval(global_model, test_loader, device)
        fedargs.tb.add_scalar("Gloabl Accuracy/", global_test_output["accuracy"], epoch)
        log.jsoninfo(global_test_output, "Gloabl Test Outut after Epoch {} of {}".format(epoch, fedargs.epochs))
        
    # Push Server Update    
    ctp.produce_model(server, fedargs.server_topic, global_model, epoch)
    
    # Clients
    tasks = [process(client, epoch, ctp, client_models[client],
                     clients_info[client]['train_loader'],
                     clients_info[client]['test_loader'],
                     fedargs, device) for client in clients]
    await asyncio.wait(tasks)

print(time.time() - start_time)

  0%|          | 0/10 [00:00<?, ?it/s]2021-08-18 16:13:54,999 - <ipython-input-20-4eda9d332668>::<module>(l:8) : Federated Training Epoch 1 of 10 [MainProcess : MainThread (INFO)]
2021-08-18 16:13:55,353 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:56) : Producing user records to topic pyflx-cs. ^C to exit. [MainProcess : MainThread (INFO)]
2021-08-18 16:13:55,636 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::produce(l:66) : Flushing records... [MainProcess : MainThread (INFO)]
2021-08-18 16:13:56,680 - /home/harsh_1921cs01/hub/F3IA/fl/libs/protobuf_producer.py::delivery_report(l:50) : User record b'bladecluster.iitp.org(server)' successfully produced to pyflx-cs [0] at offset 68 [MainProcess : MainThread (INFO)]
2021-08-18 16:13:56,741 - <ipython-input-19-89d53eeba195>::process(l:12) : Processing Client bladecluster.iitp.org(4) [MainProcess : asyncio_0 (INFO)]
2021-08-18 16:13:56,765 - <ipython-input-19-89d53eeba195>::process(l:12) : Processi